# HIRE ME please <33333333

Nvm, here is the code. Enjoy!

## Imports...

In [1]:
import requests
from pathlib import Path
import os
from tqdm.notebook import tqdm
import pandas as pd

## Getting all county ID:s

In [2]:
url = "https://stiftelser.lansstyrelsen.se/Search/GetCarouselItems"
response = requests.get(url)

all_county_ids = []

if response.ok:
    response_dict = response.json()
else:
    print("Error:", response.status_code, response.text)

for county in response_dict['items']:
    path = Path(county['imagesUrl'])
    county_name = os.path.splitext(path.name)[0]
  
    all_county_ids.append({
        "name": county_name,
        "searchLink":county['searchLink']}
        )
    
all_county_ids

[{'name': 'norrbotten', 'searchLink': '25'},
 {'name': 'orebro', 'searchLink': '18'},
 {'name': 'ostergotland', 'searchLink': '05'},
 {'name': 'skane', 'searchLink': '12'},
 {'name': 'stockholm', 'searchLink': '01'},
 {'name': 'vastra-gotaland', 'searchLink': '14'}]

## Getting all foundations for each county, and then getting all members from each foundation... 

In [ ]:
# Creating a DataFrame with following columns
df = pd.DataFrame(columns=["county", "foundation_name", "org_nbr", "person_name", "role"])

# Search word used for filtering on role
search_word = "styrelse"

# Looping over all counties... 
for county in all_county_ids:
   
  url = "https://stiftelser.lansstyrelsen.se/search/avdsearch" 
  data = {"regmyndighet": county['searchLink']}

  response = requests.post(url, json=data)

  if response.ok:
    response_dict = response.json()
    print("Scraping",county['name'])
  else:
      print("Error:", response.status_code, response.text)

  # all foundation in county...
  all_foundations = response_dict['stiftelser']

  # Looping over all foundations...
  for foundation in tqdm(all_foundations, desc="Fetching data"):

    # Get board members for each foundation
    url = "https://stiftelser.lansstyrelsen.se/search/fetchstift"
    data = {"fetchedByStiftelseID": foundation['id']}

    response = requests.post(url, json=data)

    if response.ok:
      response_dict = response.json()
    else:
        print("Error:", response.status_code, response.text)

    board_members = response_dict['roller']

    filtered_board_members = [p for p in board_members if search_word in p['typ'].lower()]

    # Looping over all members and appending a new row for each board member...
    for person in filtered_board_members:
      df.loc[len(df)] = [
        county['name'],
        foundation['namn'],
        foundation['orgnrNoMinus'][:-4] + "-" + foundation['orgnrNoMinus'][-4:],
        person['namn'],
        person['typ']
      ]

Scraping norrbotten


Fetching data:   0%|          | 0/1029 [00:00<?, ?it/s]

## Saving to csv...

In [ ]:
# 3. Export to CSV
df.to_csv("board_members.csv", index=False)